In [ ]:
## using graphviz
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot
## 아래 있는것은 필수는 아닌데, 가끔 에러가 생길 때가 있어서, 그냥 같이 해줌.
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz
import pygraphviz

In [ ]:
!pip install node2vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from node2vec import Node2Vec
import networkx as nx
import pygraphviz as pgv
import multiprocessing

dot_filename = "/content/drive/MyDrive/0-cpg.dot"  # 실제 DOT 파일의 경로

# DOT 파일 파싱 및 그래프 생성 함수
def parse_dot_file(dot_filename):
    graph = pgv.AGraph(dot_filename)
    cpg_graph = nx.DiGraph()

    for edge in graph.edges():
        source = edge[0]
        target = edge[1]
        cpg_graph.add_edge(source, target)

    return cpg_graph

# DOT 파일 파싱 및 그래프 생성
cpg_graph = parse_dot_file(dot_filename)

# Node2Vec 모델 생성 (병렬처리 비활성화)
node2vec = Node2Vec(cpg_graph, dimensions=128, walk_length=10, num_walks=10, workers=1)

# 모델 학습
model = node2vec.fit(window=5, min_count=1, batch_words=4)

# Betweenness Centrality 계산
betweenness_centrality = nx.betweenness_centrality(cpg_graph)

# 중요도가 가장 높은 노드 찾기
most_important_node = max(betweenness_centrality, key=betweenness_centrality.get)

# 중요한 노드에 대한 벡터 출력
if str(most_important_node) in model.wv:
    important_node_vector = model.wv[str(most_important_node)]
    print(f"Most important node: {most_important_node}")
    print(f"Vector: {important_node_vector}")
else:
    print("Vector not available for the most important node.")
